In [1]:
%pwd

'/workspaces/codespaces-blank/research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'/workspaces/codespaces-blank'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    
    documents=loader.load()

    return documents

In [6]:
extracted_data=load_pdf_file('Data/')

In [7]:
len(extracted_data)

637

In [10]:
print(extracted_data[48])

page_content='Description
Actinomycosis is a relatively rare infection occurring
in one out of 300,000(1/300,000) people per year. It is
characterized by the presence of a lump or mass that often
forms, draining sinus tracts to the skin surface. Fifty per-
cent of actinomycosis cases are of the head and neck region
(also called “lumpy jaw” and “cervicofacial actinomyco-
sis”), 15% are in the chest, 20% are in the abdomen, and
the rest are in the pelvis, heart, and brain. Men are three
times more likely to develop actinomycosis than women.
Causes and symptoms
Actinomycosis is usually caused by the bacterium
Actinomyces israelii. This bacterium is normally present
in the mouth but can cause disease if it enters tissues fol-
lowing an injury. Actinomyces israelii is an anaerobic
bacterium which means it dislikes oxygen but grows very
well in deep tissues where oxygen levels are low. Tooth
extraction , tooth disease, root canal treatment ,j a w
surgery, or poor dental hygiene can allow Act

In [11]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks: ", len(text_chunks))

Length of Text Chunks:  5860


In [36]:
from dotenv import load_dotenv

load_dotenv()

True

In [31]:
os.environ["MISTRAL_API_KEY"] = os.getenv("MISTRAL_API_KEY")

In [ ]:
from langchain_mistralai import MistralAIEmbeddings

embeddings = MistralAIEmbeddings(
    model="mistral-embed",

)

In [32]:
single_vector = embeddings.embed_query("hi")
print(str(single_vector)[:100])

[-0.02178955078125, -0.02880859375, 0.018035888671875, 0.006084442138671875, 0.039947509765625, 0.05


In [38]:
vector_length=len(single_vector)

In [37]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [39]:
index_name = "medical"

pc.create_index(
    name=index_name,
    dimension=vector_length, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "medical",
    "metric": "cosine",
    "host": "medical-2rz7ocg.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null
}

In [40]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 06 Apr 2025 09:22:53 GMT', 'Content-Type': 'application/json', 'Content-Length': '118', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '2287', 'x-pinecone-request-id': '7680382207863614485', 'x-envoy-upstream-service-time': '0', 'server': 'envoy'})
HTTP response body: {"code":11,"message":"Error, message length too large: found 5873759 bytes, the limit is: 4194304 bytes","details":[]}
